## Task 36-> Implement with TensorFlow/Keras
Submitted by: Awais Anwer

solution: i am going to do cat/dog classification using a dataset from kaggle

In [1]:
import cv2
import os
import numpy as np
import random
import matplotlib.pyplot as plt

In [2]:
SIZE = (170, 150)
labels_ = {'cats': 0, 'dogs': 1}

In [3]:
def load_data(path,size):
    images = []
    labels = []
    class_names = []
    
    for class_name in os.listdir(path):
        class_path = os.path.join(path, class_name)
        if os.path.isdir(class_path):
            for img_name in os.listdir(class_path):
                img_path = os.path.join(class_path, img_name)
                img = cv2.imread(img_path)
                if img is not None:
                    img = cv2.resize(img, size)  # Resize images to 150x150
                    images.append(img)
                    labels.append(labels_[class_name])
    
    images = np.array(images)
    labels = np.array(labels)
    return images, labels

In [4]:
# load training data
train_imgs,train_labels = load_data("/kaggle/input/cats-and-dogs-image-classification/train",SIZE)

In [5]:
# load test data
test_imgs,test_labels = load_data("/kaggle/input/cats-and-dogs-image-classification/test",SIZE)

In [6]:
train_imgs.shape

(557, 150, 170, 3)

In [7]:
test_imgs.shape

(140, 150, 170, 3)

In [8]:
np.unique(train_labels, return_counts=True)

(array([0, 1]), array([279, 278]))

In [9]:
# converter RGB -> GrayScale
def to_gray(images):
    grayscale_images = [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in images]
    return np.array(grayscale_images)

In [10]:
train_imgs= to_gray(train_imgs)
test_imgs= to_gray(test_imgs)

In [11]:
train_imgs.shape

(557, 150, 170)

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input,Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [22]:
input_shape = (SIZE[0], SIZE[1], 1)

# Building the model
model = Sequential()

Input(shape=input_shape)
model.add(Conv2D(32, (5, 5), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25)) 

model.add(Flatten())

# Fully connected layers
model.add(Dense(120, activation='relu'))
model.add(Dense(84, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compiling the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Display the model summary
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_10 (Conv2D)              │ (None, 166, 146, 32)   │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 83, 73, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 83, 73, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 79, 69, 64)     │        51,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 39, 34, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 39, 34, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 37, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 18, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 18, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 36864)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 120)            │     4,423,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 84)             │        10,164 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            85 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,560,001 (17.40 MB)

 Trainable params: 4,560,001 (17.40 MB)

 Non-trainable params: 0 (0.00 B)

In [19]:
train_images, val_images, train_labels_, val_labels = train_test_split(train_imgs, train_labels, test_size=0.2, random_state=42)

In [23]:
history = model.fit(train_imgs, train_labels,
                        validation_data=(test_imgs, test_labels),
                        epochs=30,
                        batch_size=64
                   ) 

Epoch 1/30
3/9 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.5538 - loss: 171.9291

W0000 00:00:1723316342.421054      87 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 746ms/step - accuracy: 0.5426 - loss: 133.7342

W0000 00:00:1723316349.026012      86 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


9/9 ━━━━━━━━━━━━━━━━━━━━ 15s 861ms/step - accuracy: 0.5402 - loss: 128.4689 - val_accuracy: 0.5000 - val_loss: 0.8117
Epoch 2/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - accuracy: 0.4946 - loss: 1.5416 - val_accuracy: 0.5143 - val_loss: 0.6929
Epoch 3/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.5317 - loss: 0.6903 - val_accuracy: 0.4929 - val_loss: 0.6931
Epoch 4/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.4823 - loss: 0.6939 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 5/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.4776 - loss: 0.6926 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 6/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.5285 - loss: 0.6917 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 7/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.5512 - loss: 0.6893 - val_accuracy: 0.5071 - val_loss: 0.6930
Epoch 8/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.5195 - loss: 0.6930 - val_accuracy: 0.5143 - val_loss: 0.6929
Epoch 9